# Preliminares

Lo que desea el organismo es saber la probabilidad de que un inviduo presente salarios por sobre o bajo los 50.000 dolares anuales, al ser dos condiciones es un problema de clasificación.

Para medir el desempeño se utilizará una matriz de confusión, donde se cruza la información de la predicción con las etiquetas reales de la muestra de validación. (lib -> sklearn.metrics import confusion_matrix

# Aspectos computacionales
pandas: Es una herramienta para analizar y manipular datos.

numpy: Es una herramienta que proporciona un objeto de matriz multidimensional y varias operaciones para matrices.

pyplot: Crea figuras, trazados, lineas, etc. En general tiene funciones para graficar

seaborn: Es una libreria de visualización de datos.

confusion_matrix: Calcula la matriz de confución para evaluar el desempeño de una clasificación.

PolynomialFeatures: Genera una matriz con las interacciones de todas las combinaciones de polinomios. 


# Datos:

Como en el enunciado avisa que los datos perdidos no son NaN, sino un '?', se utiliza el where de numpy para buscar aquellas celdas que tienen el signo y no tomarlas en cuenta. Luego para verificar se utiliza un dropna().

Para recodificar en la mayoria de columnas se utiliza la funcion loc gracias a su propiedad de acceder a una fila o columna según una condición. También para la columna 'income' se utiliza where() de numpy ya que son solo dos datos y se puede representar de forma binaria. Para verificar que los pasos anteriores fueron efectivos y que no quedaron datos perdidos, se verificará a traves de una consulta. Por ultimo, se eliminarán las columnas anteriores.

Para tener las metricas de evaluación, se va a generar tres modelos, modelo lineal aditivo, modelo lineal con interacciones de 2-orden y modelo lineal con interaccion de 3-orden, para generar las interacciones se usará PolynomialFeatures, posteriormente se concatena la función fit_transform para luego graficar el comportamiento de los datos de los tres modelos mediante validación cruzada.

In [1]:
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import PolynomialFeatures

In [16]:
df = pd.read_csv("income-db.csv")
df= df.where(df!='?')
df= df.dropna()
df

age     workclass  fnlwgt     education  educational-num  \
0       25       Private  226802          11th                7   
1       38       Private   89814       HS-grad                9   
2       28     Local-gov  336951    Assoc-acdm               12   
3       44       Private  160323  Some-college               10   
5       34       Private  198693          10th                6   
...    ...           ...     ...           ...              ...   
48837   27       Private  257302    Assoc-acdm               12   
48838   40       Private  154374       HS-grad                9   
48839   58       Private  151910       HS-grad                9   
48840   22       Private  201490       HS-grad                9   
48841   52  Self-emp-inc  287927       HS-grad                9   

           marital-status         occupation   relationship   race  gender  \
0           Never-married  Machine-op-inspct      Own-child  Black    Male   
1      Married-civ-spouse    Farming-fishing        Husband  White    Male   
2      Married-civ-spouse    Protective-serv        Husband  White    Male   
3      Married-civ-spouse  Machine-op-inspct        Husband  Black    Male   
5           Never-married      Other-service  Not-in-family  White    Male   
...                   ...                ...            ...    ...     ...   
48837  Married-civ-spouse       Tech-support           Wife  White  Female   
48838  Married-civ-spouse  Machine-op-inspct        Husband  White    Male   
48839             Widowed       Adm-clerical      Unmarried  White  Female   
48840       Never-married       Adm-clerical      Own-child  White    Male   
48841  Married-civ-spouse    Exec-managerial           Wife  White  Female   

       capital-gain  capital-loss  hours-per-week native-country income  
0                 0             0              40  United-States  <=50K  
1                 0             0              50  United-States  <=50K  
2                 0             0              40  United-States   >50K  
3              7688             0              40  United-States   >50K  
5                 0             0              30  United-States  <=50K  
...             ...           ...             ...            ...    ...  
48837             0             0              38  United-States  <=50K  
48838             0             0              40  United-States   >50K  
48839             0             0              40  United-States  <=50K  
48840             0             0              20  United-States  <=50K  
48841         15024             0              40  United-States   >50K  

[45222 rows x 15 columns]

In [3]:
#1 ← Prof-specialty, Exec-managerial, Adm-clerical, sales, Tech-support.
#2 ← Craft-repair, Machine-op-inspct, Transport-moving, Handlers-cleaners, Farming-fishing, Protective-serv, Priv-house-serv.
#3 ← Other-service, Armed-Forces.

df.loc[(df['occupation']=="Prof-specialty") | (df['occupation']=="Exec-managerial") | (df['occupation']=="Adm-clerical")| (df['occupation']=="Sales")| (df['occupation']=="Tech-support"), 'collars'] = 1


df.loc[((df['occupation']=="Craft-repair") | (df['occupation']=="Machine-op-inspct") | (df['occupation']=="Transport-moving")| (df['occupation']=="Handlers-cleaners")| (df['occupation']=="Farming-fishing") | (df['occupation']=="Protective-serv") | (df['occupation']=="Priv-house-serv")), 'collars'] = 2


df.loc[((df['occupation']=="Other-service") | (df['occupation']=="Armed-Forces")), 'collars'] = 3

df=df.drop(['occupation'],axis=1)

In [4]:
df[(df['collars']!=1)& (df['collars']!=2)&(df['collars']!=3)]

Empty DataFrame
Columns: [age, workclass, fnlwgt, education, educational-num, marital-status, relationship, race, gender, capital-gain, capital-loss, hours-per-week, native-country, income, collars]
Index: []

In [5]:
#1 ← Federal-gov.
#2 ← State-gov, Local-gov.
#3 ← Self-emp-inc, Self-emp-not-inc
#4 ← Never-worked, Without-pay.
#5 ← private

df.loc[((df['workclass']=="Federal-gov")), 'workclass_recod'] = 1

df.loc[((df['workclass']=="State-gov") | (df['workclass']=='Local-gov')), 'workclass_recod'] = 2

df.loc[((df['workclass']=="Self-emp-inc") | (df['workclass']=="Self-emp-not-inc")), 'workclass_recod'] = 3

df.loc[((df['workclass']=="Never-worked")| (df['workclass']=="Without-pay")), 'workclass_recod'] = 4

df.loc[((df['workclass']=="Private")), 'workclass_recod'] = 5

df=df.drop(['workclass'],axis=1)

In [6]:
df[(df['workclass_recod']!=1)& (df['workclass_recod']!=2)&(df['workclass_recod']!=3)&(df['workclass_recod']!=4)&(df['workclass_recod']!=5)]

Empty DataFrame
Columns: [age, fnlwgt, education, educational-num, marital-status, relationship, race, gender, capital-gain, capital-loss, hours-per-week, native-country, income, collars, workclass_recod]
Index: []

In [7]:
#1 ← Preschool
#2 ← 1st-4th, 5th-6th
#3 ← 7th-8th, 9th, 10th,11th, 12th, HS-grad
#4 ← Assoc-voc, Assoc-acdm, Some-college
#5 ← Bachelors, Masters, Prof-school, Doctorate.

df.loc[((df['education']=="Preschool")), 'educ_recod'] = 1

df.loc[((df['education']=="1st-4th")|(df['education']=="5th-6th")), 'educ_recod'] = 2

df.loc[((df['education']=="7th-8th")|(df['education']=="9th")|(df['education']=="10th")|(df['education']=="11th")|(df['education']=="12th")|(df['education']=="HS-grad")), 'educ_recod'] = 3

df.loc[((df['education']=="Assoc-voc")|(df['education']=="Assoc-acdm")|(df['education']=="Some-college")), 'educ_recod'] = 4

df.loc[((df['education']=="Bachelors")|(df['education']=="Masters")|(df['education']=="Prof-school")|(df['education']=="Doctorate")), 'educ_recod'] = 5

df=df.drop(['education'],axis=1)

In [8]:
df[(df['educ_recod']!=1)& (df['educ_recod']!=2)&(df['educ_recod']!=3)&(df['educ_recod']!=4)&(df['educ_recod']!=5)]

Empty DataFrame
Columns: [age, fnlwgt, educational-num, marital-status, relationship, race, gender, capital-gain, capital-loss, hours-per-week, native-country, income, collars, workclass_recod, educ_recod]
Index: []

In [9]:
#1 ← Married-civ-spouse, Married-spouse-absent, Married-AF-spouse.
#2 ← Divorced.
#3 ← Separated.
#4 ← Widowed.

df.loc[((df['marital-status']=="Married-civ-spouse")|(df['marital-status']=="Married-spouse-absent")|(df['marital-status']=="Married-AF-spouse")), 'civstatus'] = 1

df.loc[(df['marital-status']=="Divorced"), 'civstatus'] = 2

df.loc[(df['marital-status']=="Separated"), 'civstatus'] = 3

df.loc[(df['marital-status']=="Widowed"), 'civstatus'] = 4

df.loc[(df['marital-status']=="Never-married"), 'civstatus'] = 5

df=df.drop(['marital-status'],axis=1)

In [10]:
df[(df['civstatus']!=1)& (df['civstatus']!=2)& (df['civstatus']!=3)&(df['civstatus']!=4)&(df['civstatus']!=5)]

Empty DataFrame
Columns: [age, fnlwgt, educational-num, relationship, race, gender, capital-gain, capital-loss, hours-per-week, native-country, income, collars, workclass_recod, educ_recod, civstatus]
Index: []

In [11]:
#native-country: País de origen. United-States, Cambodia, England, Puerto-Rico,
#Canada, Germany, Outlying-US(Guam-USVI-etc), India, Japan, Greece, South, China,
#Cuba, Iran, Honduras, Philippines, Italy, Poland, Jamaica, Vietnam, Mexico, Portugal,
#Ireland, France, Dominican-Republic, Laos, Ecuador, Taiwan, Haiti, Columbia,
#Hungary, Guatemala, Nicaragua, Scotland, Thailand, Yugoslavia, El-Salvador,
#Trinadad&Tobago, Peru, Hong, Holand-Netherlands.
#america
df.loc[((df['native-country']=="United-States")|(df['native-country']=="Puerto-Rico"))|(df['native-country']=="Canada")|(df['native-country']=="Cuba")|(df['native-country']=="Honduras")|(df['native-country']=="Jamaica")|(df['native-country']=="Mexico")|(df['native-country']=="Dominican-Republic")|(df['native-country']=="Ecuador")|(df['native-country']=="Haiti")|(df['native-country']=="Columbia")|(df['native-country']=="Guatemala")|(df['native-country']=="Nicaragua")|(df['native-country']=="El-Salvador")|(df['native-country']=="Trinadad&Tobago")|(df['native-country']=="Peru"), 'region']=1

#asia
df.loc[(df['native-country']=="Cambodia")|(df['native-country']=="India")|(df['native-country']=="Japan")|(df['native-country']=="China")|(df['native-country']=="Iran")|(df['native-country']=="Philippines")|(df['native-country']=="Vietnam")|(df['native-country']=="Laos")|(df['native-country']=="Taiwan")|(df['native-country']=="Thailand")|(df['native-country']=="Hong"), 'region'] = 2

#europa
df.loc[(df['native-country']=="England")|(df['native-country']=="Germany")|(df['native-country']=="Greece")|(df['native-country']=="Italy")|(df['native-country']=="Poland")|(df['native-country']=="Portugal")|(df['native-country']=="Ireland")|(df['native-country']=="France")|(df['native-country']=="Portugal")|(df['native-country']=="Hungary")|(df['native-country']=="Scotland")|(df['native-country']=="Yugoslavia")|(df['native-country']=="Holand-Netherlands"), 'region'] = 3

#africa
df.loc[(df['native-country']=="South"), 'region'] = 4

#aceania
df.loc[((df['native-country']=="Outlying-US(Guam-USVI-etc)")), 'region'] = 5

df=df.drop(['native-country'],axis=1)

In [12]:
df

age  fnlwgt  educational-num   relationship   race  gender  \
0       25  226802                7      Own-child  Black    Male   
1       38   89814                9        Husband  White    Male   
2       28  336951               12        Husband  White    Male   
3       44  160323               10        Husband  Black    Male   
5       34  198693                6  Not-in-family  White    Male   
...    ...     ...              ...            ...    ...     ...   
48837   27  257302               12           Wife  White  Female   
48838   40  154374                9        Husband  White    Male   
48839   58  151910                9      Unmarried  White  Female   
48840   22  201490                9      Own-child  White    Male   
48841   52  287927                9           Wife  White  Female   

       capital-gain  capital-loss  hours-per-week income  collars  \
0                 0             0              40  <=50K      2.0   
1                 0             0              50  <=50K      2.0   
2                 0             0              40   >50K      2.0   
3              7688             0              40   >50K      2.0   
5                 0             0              30  <=50K      3.0   
...             ...           ...             ...    ...      ...   
48837             0             0              38  <=50K      1.0   
48838             0             0              40   >50K      2.0   
48839             0             0              40  <=50K      1.0   
48840             0             0              20  <=50K      1.0   
48841         15024             0              40   >50K      1.0   

       workclass_recod  educ_recod  civstatus  region  
0                  5.0         3.0        5.0     1.0  
1                  5.0         3.0        1.0     1.0  
2                  2.0         4.0        1.0     1.0  
3                  5.0         4.0        1.0     1.0  
5                  5.0         3.0        5.0     1.0  
...                ...         ...        ...     ...  
48837              5.0         4.0        1.0     1.0  
48838              5.0         3.0        1.0     1.0  
48839              5.0         3.0        4.0     1.0  
48840              5.0         3.0        5.0     1.0  
48841              3.0         3.0        1.0     1.0  

[45222 rows x 15 columns]

In [13]:
df['Income']=np.where(df['income']=='<=50K',1,0)
df=df.drop(['income'],axis=1)

In [14]:
df

age  fnlwgt  educational-num   relationship   race  gender  \
0       25  226802                7      Own-child  Black    Male   
1       38   89814                9        Husband  White    Male   
2       28  336951               12        Husband  White    Male   
3       44  160323               10        Husband  Black    Male   
5       34  198693                6  Not-in-family  White    Male   
...    ...     ...              ...            ...    ...     ...   
48837   27  257302               12           Wife  White  Female   
48838   40  154374                9        Husband  White    Male   
48839   58  151910                9      Unmarried  White  Female   
48840   22  201490                9      Own-child  White    Male   
48841   52  287927                9           Wife  White  Female   

       capital-gain  capital-loss  hours-per-week  collars  workclass_recod  \
0                 0             0              40      2.0              5.0   
1                 0             0              50      2.0              5.0   
2                 0             0              40      2.0              2.0   
3              7688             0              40      2.0              5.0   
5                 0             0              30      3.0              5.0   
...             ...           ...             ...      ...              ...   
48837             0             0              38      1.0              5.0   
48838             0             0              40      2.0              5.0   
48839             0             0              40      1.0              5.0   
48840             0             0              20      1.0              5.0   
48841         15024             0              40      1.0              3.0   

       educ_recod  civstatus  region  Income  
0             3.0        5.0     1.0       1  
1             3.0        1.0     1.0       1  
2             4.0        1.0     1.0       0  
3             4.0        1.0     1.0       0  
5             3.0        5.0     1.0       1  
...           ...        ...     ...     ...  
48837         4.0        1.0     1.0       1  
48838         3.0        1.0     1.0       0  
48839         3.0        4.0     1.0       1  
48840         3.0        5.0     1.0       1  
48841         3.0        1.0     1.0       0  

[45222 rows x 15 columns]